In [2]:
import numpy as np
import pandas as pd
# import nltk
from nltk.tokenize import sent_tokenize
# import spacy 

# sp = spacy.load("en_core_web_sm", disable=['parser', 'ner'])
# all_stopwords = sp.Defaults.stop_words

def tokenize_split(text):
    """simple tokeniser"""
    return text.split()


def word_chunk_transcript(transcripts, name_variable='transcript', chunk_size=500):
    """ transcripts: must have two columns - episode_id and tokenised_transcript
        chunk_size: number of tokens in chunk 
    """
    transcripts["tokenised_transcript"] = transcripts[name_variable].apply(lambda x: tokenize_split(x))

    episode_ls = []
    transcript_ls = []
    words_enum_ls = [] 
    for index, row in transcripts.iterrows():
        for i in range(0, len(row["tokenised_transcript"]), chunk_size):
            episode_ls.append(row["episode_id"])
            transcript_ls.append(" ".join(row["tokenised_transcript"][i:i+chunk_size]))
            words_enum_ls.append(f"{i+1} - {i+chunk_size}")
    word_chunked_df = pd.DataFrame(data = {'episode_id': episode_ls, 'transcript_subset':transcript_ls, 'words_enumerated':words_enum_ls})
    return word_chunked_df


def sentence_chunk_transcript(transcripts, name_variable='transcript', chunk_size=1):
    """ transcripts: must have two columns - episode_id and transcript
        chunk_size: number of sentences in chunk 
    """
    transcripts["sentence_token"] = transcripts[name_variable].apply(lambda x: sent_tokenize(x, language='english'))
    episode_ls = []
    transcript_ls = []
    sent_enum_ls = []
    for index, row in transcripts.iterrows():
        for i in range(0, len(row["sentence_token"]), chunk_size):
            episode_ls.append(row["episode_id"])
            transcript_ls.append(" ".join(row["sentence_token"][i:i+chunk_size]))
            sent_enum_ls.append(f"{i+1} - {i+chunk_size}")
    sentence_chunked_df = pd.DataFrame(data = {'episode_id': episode_ls, 'transcript_subset':transcript_ls, 'sentence_enumerated':sent_enum_ls})
    return sentence_chunked_df

# # Define a function to remove stopwords from a string
# def remove_stopwords(text):    
#     doc = text.split(' ')
#     return ' '.join([word for word in doc if word not in all_stopwords])

### Make chunked dataset

In [3]:
transcripts_sample = pd.read_csv('sports_transcripts.csv.gz', compression='gzip')
print(transcripts_sample.shape)
# transcripts_sample.head(5)

(11821, 17)


In [4]:
# Extract columns for conversion
cols_subset = transcripts_sample.loc[: ,["episode_id", "transcript"]]

### Create new datasets

In [5]:
sports_256 = word_chunk_transcript(cols_subset, name_variable='transcript', chunk_size=256)

print(sports_256.shape)
sports_256.head(2)

(368835, 3)


,episode_id,transcript_subset,words_enumerated
0,41JbXYp7c2uuJoFB4TcQtD,Hello and welcome to the law review podcast. M...,0 - 256
1,41JbXYp7c2uuJoFB4TcQtD,"as well as several other changes, so we'll get...",256 - 512


In [6]:
sports_256.to_csv('sports_word_256.csv.gz', compression='gzip')

In [7]:
sport_sent_1 = sentence_chunk_transcript(cols_subset, name_variable='transcript', chunk_size=1)
print(sport_sent_1.shape)
sport_sent_1.head(2)

(5460190, 3)


,episode_id,transcript_subset,sentence_enumerated
0,41JbXYp7c2uuJoFB4TcQtD,Hello and welcome to the law review podcast.,0 - 1
1,41JbXYp7c2uuJoFB4TcQtD,My name is Nathan Church Droid by my partner c...,1 - 2


In [8]:
sport_sent_1 = sport_sent_1.sample(500000, random_state=42)
sport_sent_1.shape

(500000, 3)

In [9]:
sport_sent_1.to_csv('sport_sent_1.csv.gz', compression='gzip')